
# InfoNCE 在对比学习中的应用

> **摘要**：InfoNCE 作为对比学习的核心损失函数，在 SimCLR、MoCo、CLIP 等模型中发挥着关键作用。本文将深入解析 InfoNCE 的数学原理，推导其与交叉熵的关系，以 CLIP 模型为例展示其实际应用，并提供 CLIP 风格的官方伪代码实现。

---

## 1. InfoNCE 公式及基本内容

### 1.1 什么是 InfoNCE？

InfoNCE（Information Noise Contrastive Estimation）是对比学习中的核心损失函数，其核心思想是：**在一个包含正样本和负样本的集合中，让模型能够识别出唯一的正样本**。

### 1.2 InfoNCE 数学公式

对于一个 query $q$，一个正样本 $k^+$，以及 $K$ 个负样本 $\{k^-_i\}_{i=1}^K$，InfoNCE 损失定义为：

$$
\mathcal{L}_{\text{InfoNCE}} = -\log \frac{\exp(sim(q, k^+) / \tau)}{\exp(sim(q, k^+) / \tau) + \sum_{i=1}^{K} \exp(sim(q, k^-_i) / \tau)}
$$

其中：
- $sim(q, k)$ 是相似度函数（通常为点积或余弦相似度）
- $\tau$ 是温度系数，控制分布的锐度
- 分母是所有候选样本（正样本 + 负样本）的归一化项

这个公式的核心思想是：**最大化正样本相对于所有候选样本的条件概率**。

---

## 2. InfoNCE 与交叉熵函数的关系推导

InfoNCE 实际上是交叉熵损失在对比学习场景下的具体应用。让我们通过数学推导来理解这一关系：

### Step 1: Softmax 函数
对于 logits 向量 $\mathbf{z} = [z_1, z_2, ..., z_M]$，第 $i$ 个类别的预测概率为：
$$
\hat{y}_i = \text{Softmax}(z)_i = \frac{e^{z_i}}{\sum_{j=1}^M e^{z_j}}
$$

### Step 2: 交叉熵损失
假设真实标签是 One-hot 编码，其中只有第 $k$ 个类别为 1，其余为 0，则损失为：
$$
\mathcal{L} = -\sum_{i=1}^M y_i \log(\hat{y}_i) = -\log(\hat{y}_k)
$$

所以，在one hot编码中，交叉熵损失的目标就是最大化正确分类位置的概率，即最大化 $\hat{y}_k$。

### Step 3: 将 Softmax 代入交叉熵
将第一步的 Softmax 公式代入第二步的 $\hat{y}_k$ 中：
$$
\mathcal{L} = -\log \left( \frac{e^{z_k}}{\sum_{j=1}^M e^{z_j}} \right)
$$

### Step 4: 映射到 InfoNCE 场景
在 InfoNCE 中：
- $M = K + 1$（1 个正样本 + K 个负样本）
- $z_k = sim(q, k^+) / \tau$（正样本相似度）
- $z_{\text{others}} = \{sim(q, k^-_i) / \tau\}_{i=1}^K$（负样本相似度）
- 真实标签: 正样本所在的索引位置（意味着从正负样本集合中正确分类出正样本所在的位置）

代入得到：
$$
\mathcal{L}_{\text{InfoNCE}} = -\log \left( \frac{\exp(sim(q, k^+) / \tau)}{\exp(sim(q, k^+) / \tau) + \sum_{i=1}^{K} \exp(sim(q, k^-_i) / \tau)} \right)
$$

✅ **这正是 InfoNCE 的标准公式！**

因此，**InfoNCE 本质上是在 K+1 个候选中识别正样本的交叉熵损失**。

因此，InfoNCE就是当我们把正样本和负样本的相似度计算出来之后，正确分类出正样本的位置。因此label就是正样本的位置，比如正样本是第0个，那么label就是0，正样本是第1个，那么label就是1，以此类推。

---

## 3. CLIP 模型如何使用 InfoNCE

CLIP（Contrastive Language–Image Pre-training）是 InfoNCE 在多模态学习中的成功应用，通过大规模图像-文本对训练实现了强大的零样本迁移能力。

### 3.1 模型架构
- **图像编码器** $f_I$：将图像 $I$ 映射为向量 $v$
- **文本编码器** $f_T$：将文本 $T$ 映射为向量 $t$
- 两个编码器独立训练，但共享对比学习目标

### 3.2 训练流程
给定一个 batch 的 $N$ 个图像-文本对 $\{(I_i, T_i)\}_{i=1}^N$：

1. **编码**：$v_i = f_I(I_i)$, $t_i = f_T(T_i)$
2. **归一化**：$v_i \leftarrow \frac{v_i}{\|v_i\|_2}$, $t_i \leftarrow \frac{t_i}{\|t_i\|_2}$
3. **相似度计算**：$S = V \cdot T^T$，其中 $V = [v_1, ..., v_N]^T$，$T = [t_1, ..., t_N]^T$
4. **双向对比损失**：

**图像到文本**：
$$
\mathcal{L}_{\text{img2txt}} = -\frac{1}{N} \sum_{i=1}^N \log \frac{\exp(S_{ii} / \tau)}{\sum_{j=1}^N \exp(S_{ij} / \tau)}
$$

**文本到图像**：
$$
\mathcal{L}_{\text{txt2img}} = -\frac{1}{N} \sum_{i=1}^N \log \frac{\exp(S_{ii} / \tau)}{\sum_{j=1}^N \exp(S_{ji} / \tau)}
$$

**总损失**：$\mathcal{L}_{\text{CLIP}} = \frac{1}{2}(\mathcal{L}_{\text{img2txt}} + \mathcal{L}_{\text{txt2img}})$

### 3.3 关键特点
- **对称性**：双向对比确保图像和文本空间对齐
- **零样本能力**：训练后可直接用于图像分类、检索等任务
- **大规模数据**：利用 4 亿图像-文本对进行训练

---

## 4. CLIP 伪代码


```python
# f_I: image encoder (e.g., ViT or ResNet)
# f_T: text encoder (e.g., Transformer)
# τ: temperature (learnable or fixed)
# B: batch size

for (images, texts) in dataloader:      # load B image-text pairs
    # Encode images and texts
    img_features = f_I(images)          # [B, D]
    text_features = f_T(texts)          # [B, D]

    # L2 normalize (for cosine similarity)
    img_features = normalize(img_features, dim=1)
    text_features = normalize(text_features, dim=1)

    # Compute similarity matrix
    logits_per_image = (img_features @ text_features.T) / τ   # [B, B]
    logits_per_text = logits_per_image.T                       # [B, B]

    # Labels: diagonal elements are positive pairs
    labels = arange(B)                  # [B], e.g., [0, 1, 2, ..., B-1]

    # Compute symmetric loss
    loss_img2txt = CrossEntropyLoss(logits_per_image, labels)
    loss_txt2img = CrossEntropyLoss(logits_per_text, labels)

    total_loss = (loss_img2txt + loss_txt2img) / 2

    # Update both encoders
    total_loss.backward()
    optimizer.step()
```

### 代码说明：
- `logits_per_image[i, j]` 表示第 i 个图像与第 j 个文本的相似度
- 对角线元素 `logits[i, i]` 是正样本对的相似度
- `labels = arange(B)` 表示每个图像的正确文本是对应位置的
- 双向损失确保图像和文本编码器都得到充分训练
- clip模型是使用一个optimzer`同时更新两个编码器`。

---



## 5. Batch Size 的挑战与 MoCo 的解决方案
对比学习本质上是想让正样本的互信息最大化。理论上，负样本越多，对整个数据分布（文本空间/图像空间）的模拟就越准确。

在 CLIP 的 InfoNCE 实现中，**负样本完全来自于当前 batch 中的其他样本**。这意味着：
- 负样本数量 = `batch_size - 1`
- 要获得大量负样本（如 K=32768），就必须使用极大的 batch size（如 32769）

OpenAI 在训练 CLIP 时正是采用了 **超大 batch size（高达 32768）**，配合大规模 GPU 集群实现。这带来了两个问题：
1. **显存限制**：普通研究者或开发者难以负担如此大的显存开销；
2. **硬件门槛高**：需要多卡同步、梯度累积等复杂工程支持。

### 那么，有没有办法在小 batch 下依然使用大量负样本？

答案是：**有！MoCo（Momentum Contrast）提供了一种优雅的解决方案。**

### MoCo 的核心思想

MoCo 的关键洞察是：**解耦负样本数量 $K$ 与 batch size $N$**。它通过引入一个**动态字典（dynamic dictionary）** 来存储历史 key，具体包括两个创新：

1. **队列（Queue）机制**  
   维护一个大小为 $K$ 的先进先出（FIFO）队列，用于存储过去 batch 产生的 key。这样，即使 batch size 很小（如 N=256），也能拥有 K=65536 个负样本。

2. **动量编码器（Momentum Encoder）**  
   使用一个缓慢更新的 key 编码器 $f_k$（参数 $\theta_k$），其更新方式为：
   $$
   \theta_k \leftarrow m \cdot \theta_k + (1 - m) \cdot \theta_q
   $$
   其中 $m \approx 0.999$。这保证了队列中 key 的表示一致性，避免因编码器剧烈变化导致负样本“语义漂移”。

> 💡 简单说：**MoCo 用“时间换空间”——用历史样本代替同 batch 样本作为负例。**

---

## 6. MoCo 伪代码


```python
# f_q, f_k: encoder networks for query and key
# queue: dictionary as a queue of K keys (CxK)
# m: momentum coefficient (e.g., 0.999)
# t: temperature (e.g., 0.07)

f_k.params = f_q.params  # initialize key encoder with query encoder

for x in loader:  # load a minibatch x with N samples
    x_q = aug(x)  # a randomly augmented version
    x_k = aug(x)  # another randomly augmented version

    q = f_q.forward(x_q)  # queries: NxC
    k = f_k.forward(x_k)  # keys: NxC
    k = k.detach()        # no gradient to keys

    # positive logits: Nx1
    l_pos = bmm(q.view(N,1,C), k.view(N,C,1))
    # negative logits: NxK
    l_neg = mm(q.view(N,C), queue.view(C,K))
    # logits: Nx(1+K)
    logits = cat([l_pos, l_neg], dim=1)

    # contrastive loss
    labels = zeros(N)  # positives are the 0-th
    loss = CrossEntropyLoss(logits / t, labels)

    # SGD update: query network
    loss.backward()
    update(f_q.params)

    # momentum update: key network
    f_k.params = m * f_k.params + (1 - m) * f_q.params

    # update dictionary
    enqueue(queue, k)   # add current keys to queue
    dequeue(queue)      # remove oldest if exceeds size K
```

### 对比 CLIP 与 MoCo：
| 特性 | CLIP | MoCo |
|------|------|------|
| 负样本来源 | 同 batch 其他样本 | 历史 batch（队列）|
| 负样本数量 | = batch_size - 1 | 固定 K（如 65536）|
| 是否需大 batch | 是 | 否 |
| 编码器数量 | 2（图像/文本）| 2（query/key）|
| 适用场景 | 多模态对齐 | 单模态自监督 |

---

## 7. 总结

其实，InfoNCE 的本质超简单：

👉 它就是一个 (K+1) 分类任务 —— 给你一个图像（query），再给 K+1 个候选（1个正样本 + K个负样本），让模型找出“谁和它是一对”。

这个分类的目标就是正样本所在位置的one hot编码。调用pytorch的cross entropy就是传入正样本的索引。

InfoNCE 是一个`“1 正 + N 负”`的多分类任务，总类别数 = N+1。在 **MoCo** 中 `N=K`（队列大小），在 **CLIP** 中 `N=B−1`（batch 内其他样本）

---


InfoNCE 作为对比学习的核心，通过巧妙的数学设计实现了高效的学习机制：

1. **理论基础**：InfoNCE 本质上是交叉熵损失的特殊形式，将对比学习转化为分类问题
2. **实用价值**：在 SimCLR、MoCo、CLIP 等模型中取得了显著效果
3. **扩展性**：支持同模态（图像-图像）和跨模态（图像-文本）对比学习
4. **未来前景**：为多模态大模型、自监督学习等领域提供了重要技术支撑

通过深入理解 InfoNCE，我们可以更好地掌握现代对比学习的核心原理，并为相关研究和应用提供坚实的理论基础。
